## Introdução à Regressão - Aula 1

#### Colocando em Perspectiva

✅ Existem muitos tipos de métodos de regressão, e qual você escolhe depende da resposta que está procurando. Se quiser prever a altura provável de uma pessoa com uma determinada idade, utilizaria `regressão linear`, pois está à procura de um **valor numérico**. Se estiver interessado em descobrir se um tipo de culinária deve ser considerado vegano ou não, está à procura de uma **atribuição de categoria**, então utilizaria `regressão logística`. Aprenderá mais sobre regressão logística mais adiante. Pense um pouco sobre algumas perguntas que pode fazer aos dados e qual desses métodos seria mais apropriado.

Nesta seção, irá trabalhar com um [pequeno conjunto de dados sobre diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Imagine que queria testar um tratamento para pacientes diabéticos. Modelos de Machine Learning podem ajudar a determinar quais pacientes responderiam melhor ao tratamento, com base em combinações de variáveis. Mesmo um modelo de regressão muito básico, quando visualizado, pode mostrar informações sobre variáveis que ajudariam a organizar os seus ensaios clínicos teóricos.

Dito isso, vamos começar esta tarefa!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Arte por @allison_horst</figcaption>

<!--![Arte por \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.pt.jpg)<br>Arte por @allison_horst-->


## 1. Carregar o nosso conjunto de ferramentas

Para esta tarefa, vamos precisar dos seguintes pacotes:

-   `tidyverse`: O [tidyverse](https://www.tidyverse.org/) é uma [coleção de pacotes R](https://www.tidyverse.org/packages) projetada para tornar a ciência de dados mais rápida, fácil e divertida!

-   `tidymodels`: O [tidymodels](https://www.tidymodels.org/) é uma [coleção de pacotes](https://www.tidymodels.org/packages/) para modelagem e aprendizagem automática.

Pode instalá-los com o seguinte comando:

`install.packages(c("tidyverse", "tidymodels"))`

O script abaixo verifica se tem os pacotes necessários para completar este módulo e instala-os caso algum esteja em falta.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Agora, vamos carregar estes pacotes incríveis e torná-los disponíveis na nossa sessão atual de R. (Isto é apenas para ilustração, `pacman::p_load()` já fez isso por si)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. O conjunto de dados sobre diabetes

Neste exercício, vamos demonstrar nossas habilidades de regressão fazendo previsões com um conjunto de dados sobre diabetes. O [conjunto de dados sobre diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) inclui `442 amostras` de dados relacionados à diabetes, com 10 variáveis preditoras, `idade`, `sexo`, `índice de massa corporal`, `pressão arterial média` e `seis medições de soro sanguíneo`, além de uma variável de resultado `y`: uma medida quantitativa da progressão da doença um ano após a linha de base.

|Número de observações|442|
|----------------------|:---|
|Número de preditores|As primeiras 10 colunas são preditivas numéricas|
|Resultado/Alvo|A coluna 11 é uma medida quantitativa da progressão da doença um ano após a linha de base|
|Informação sobre os preditores|- idade em anos
||- sexo
||- bmi índice de massa corporal
||- bp pressão arterial média
||- s1 tc, colesterol sérico total
||- s2 ldl, lipoproteínas de baixa densidade
||- s3 hdl, lipoproteínas de alta densidade
||- s4 tch, colesterol total / HDL
||- s5 ltg, possivelmente logaritmo do nível de triglicerídeos no soro
||- s6 glu, nível de açúcar no sangue|



> 🎓 Lembre-se, isto é aprendizagem supervisionada, e precisamos de um alvo chamado 'y'.

Antes de poder manipular dados com R, é necessário importar os dados para a memória do R ou estabelecer uma conexão com os dados que o R possa usar para acessá-los remotamente.

> O pacote [readr](https://readr.tidyverse.org/), que faz parte do Tidyverse, oferece uma maneira rápida e amigável de ler dados retangulares no R.

Agora, vamos carregar o conjunto de dados sobre diabetes fornecido neste URL de origem: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Além disso, faremos uma verificação básica nos nossos dados usando `glimpse()` e exibiremos as primeiras 5 linhas usando `slice()`.

Antes de avançarmos, vamos também introduzir algo que você encontrará frequentemente no código R 🥁🥁: o operador pipe `%>%`

O operador pipe (`%>%`) realiza operações em sequência lógica, passando um objeto adiante para uma função ou expressão de chamada. Você pode pensar no operador pipe como dizendo "e então" no seu código.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` mostra-nos que estes dados têm 442 linhas e 11 colunas, sendo todas as colunas do tipo de dados `double`.

<br>

> glimpse() e slice() são funções do [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, parte do Tidyverse, é uma gramática de manipulação de dados que fornece um conjunto consistente de verbos para ajudar a resolver os desafios mais comuns de manipulação de dados.

<br>

Agora que temos os dados, vamos focar numa única característica (`bmi`) como alvo para este exercício. Isto exigirá que selecionemos as colunas desejadas. Então, como fazemos isso?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) permite-nos *selecionar* (e opcionalmente renomear) colunas num data frame.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Dados de Treino e Teste

É uma prática comum em aprendizagem supervisionada *dividir* os dados em dois subconjuntos: um conjunto (geralmente maior) para treinar o modelo e um conjunto menor "reservado" para verificar como o modelo se comportou.

Agora que temos os dados prontos, podemos verificar se uma máquina pode ajudar a determinar uma divisão lógica entre os números neste conjunto de dados. Podemos usar o pacote [rsample](https://tidymodels.github.io/rsample/), que faz parte do framework Tidymodels, para criar um objeto que contém as informações sobre *como* dividir os dados, e depois usar mais duas funções do rsample para extrair os conjuntos de treino e teste criados:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Treinar um modelo de regressão linear com Tidymodels

Agora estamos prontos para treinar o nosso modelo!

No Tidymodels, especifica-se os modelos utilizando `parsnip()` ao definir três conceitos:

-   O **tipo** de modelo diferencia modelos como regressão linear, regressão logística, modelos de árvores de decisão, entre outros.

-   O **modo** do modelo inclui opções comuns como regressão e classificação; alguns tipos de modelo suportam ambos, enquanto outros têm apenas um modo.

-   O **motor** do modelo é a ferramenta computacional que será usada para ajustar o modelo. Muitas vezes, são pacotes R, como **`"lm"`** ou **`"ranger"`**

Estas informações sobre o modelo são capturadas numa especificação de modelo, então vamos construir uma!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Depois de um modelo ter sido *especificado*, pode ser `estimado` ou `treinado` utilizando a função [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), normalmente com uma fórmula e alguns dados.

`y ~ .` significa que ajustaremos `y` como a quantidade/variável alvo prevista, explicada por todos os preditores/características, ou seja, `.` (neste caso, temos apenas um preditor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

A partir do resultado do modelo, podemos observar os coeficientes aprendidos durante o treino. Eles representam os coeficientes da linha de melhor ajuste que nos proporciona o menor erro geral entre a variável real e a prevista.
<br>

## 5. Fazer previsões no conjunto de teste

Agora que treinámos um modelo, podemos utilizá-lo para prever a progressão da doença y para o conjunto de dados de teste usando [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Isto será usado para traçar a linha entre os grupos de dados.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 Acabámos de treinar um modelo e utilizámo-lo para fazer previsões!

Ao fazer previsões, a convenção do tidymodels é sempre produzir um tibble/data frame de resultados com nomes de colunas padronizados. Isto facilita a combinação dos dados originais com as previsões num formato utilizável para operações subsequentes, como a criação de gráficos.

`dplyr::bind_cols()` une eficientemente várias data frames por coluna.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Representar os resultados do modelo

Agora é hora de ver isto visualmente 📈. Vamos criar um gráfico de dispersão com todos os valores de `y` e `bmi` do conjunto de teste e, em seguida, usar as previsões para desenhar uma linha no lugar mais apropriado, entre os agrupamentos de dados do modelo.

O R tem vários sistemas para criar gráficos, mas o `ggplot2` é um dos mais elegantes e versáteis. Este permite compor gráficos **combinando componentes independentes**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Pensa um pouco sobre o que está a acontecer aqui. Uma linha reta está a passar por muitos pequenos pontos de dados, mas o que está realmente a fazer? Consegues perceber como deverias ser capaz de usar esta linha para prever onde um novo ponto de dados, ainda não visto, deveria encaixar em relação ao eixo y do gráfico? Tenta expressar em palavras a utilidade prática deste modelo.

Parabéns, construíste o teu primeiro modelo de regressão linear, criaste uma previsão com ele e exibiste-a num gráfico!



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante ter em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
